Assignment designed by Sébastien Lemieux, from the Université de Montréal
<br>
All answers were written by Alya Zeinaty
<br> Date: July 2022

(The spirit when programming: "*Why spend 5 days doing some work when you can spend 5 weeks automating it!*")

# Creating your own kernel / environment

Create a Python environment (where packages are installed):
- Open a terminal
- `python -m venv bioinfo`
- `source bioinfo/bin/activate`

Link it to a Jupyter kernel (so that your code execute with it):
- `pip install ipykernel`
- `python -m ipykernel install --user --name bioinfo --display-name "Python (bioinfo)"`

Then you can install new packages:
- `pip install pandas`

In your notebook, you'll need to `Kernel` -> `Change kernel` to this new kernel. It is probably safe to then `Kernel` -> `Restart & clear output`. Make sure this new kernel is the one active (upper right corner of your screen, just below the button `Control Panel`).

# Working with real data

In [54]:
import pandas as pd

## Read from / write to TSV and CSV files (in and out of Excel / R)

(Doc: https://pandas.pydata.org/docs/reference/api/pandas.read_table.html#pandas.read_table)

In [55]:
df=pd.read_csv("kmers.tsv", sep='\t')

(Doc: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [56]:
df.head()

,Seq,Id,Count
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,113422
1,CAGGACTCCAATATAGAGATAAGTTAATGTC,2,93
2,TATGTAATTGGTTCCAGTGTGAGTCATTAAA,3,5
3,GATATTTTCGAAAAGTGGGATTTTTTAAACC,4,88
4,CTCCATCTCAGGTATTAGAATGAATGCTTAC,5,7


## Dataframe manipulation

(.head(), .tail(), .shape, .Col, sum(), len(), .describe(), ["Col"], .drop())

In [57]:
df.shape

(4000, 3)

(Arithmetics on columns)

## Guided exercise(s) here...

### 1) Add a column with nucleotide count (A)
([loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)[*row*,*col*], .[apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)())

In [58]:
def count_nuc(seq, nuc):
    a=seq.count(nuc)
    return a

(First step: We'll try to first apply it to the second row, counting As only)

In [59]:
print(count_nuc('AAAAATTTTT', 'A'))

5


(Doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)

In [60]:
#Adding the A_count column to df
A_count=[count_nuc(seq, 'A') for seq in df.Seq]
df['A_count']=A_count
df.head()

,Seq,Id,Count,A_count
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,113422,31
1,CAGGACTCCAATATAGAGATAAGTTAATGTC,2,93,12
2,TATGTAATTGGTTCCAGTGTGAGTCATTAAA,3,5,9
3,GATATTTTCGAAAAGTGGGATTTTTTAAACC,4,88,10
4,CTCCATCTCAGGTATTAGAATGAATGCTTAC,5,7,9


(Doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

(Whiz-kid corner: lambda expressions, https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)

In [61]:
df.apply(lambda row: count_nuc(row.Seq, 'A'), axis=1)

0       31
1       12
2        9
3       10
4        9
        ..
3995     9
3996     6
3997     7
3998     9
3999    10
Length: 4000, dtype: int64

### 2) Show the 10 sequences with the most number of A. How many reads do they represent? What % of the (truncated) transcriptome?
(.sort_values())

(Doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

In [62]:
sorted_df=df.sort_values(by='A_count', ascending=False)

In [66]:
first_ten_most_As=sorted_df.head(10)
#first_ten_most_As

In [67]:
#Ten sequences with the most number of As:
print("The ten sequences with the most number of As are:")
print(first_ten_most_As.Seq)

The ten sequences with the most number of As are:
0       AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
650     CAAAAAAAAAAAAACAAAAAACAAAAAAACA
2507    AAATAACAAAAAATTAAAAAAAAAAAAAAAA
168     AAAAAAGATTAAAAAATTAAAAAAAAAAGAA
3678    AAAACAAAAACAAAACAAACAAACAAAAAAG
3880    AAAGAAAGAAAAAGAAAAAAAAAATAGCACA
2321    AATAACAGAAAGAAAACAAAAAGAAAAATAA
1663    AAAACAAACAAAAACAAACAAGAAAGAAATC
3186    ATAAATAAAAAGGAAAAGAAAAGAAAAGAAG
3491    AAAAGAAGACAAAAGAAAAGAGAAAGAAGAA
Name: Seq, dtype: object


In [68]:
#number of reads represented
print("The number of reads they represent is " +str(first_ten_most_As.Count.sum())+".")

The number of reads they represent is 113579.


In [69]:
#percentage of the transcriptome
print("The percentage of the transcriptome they represent is "+str(first_ten_most_As.Count.sum()/df.Count.sum()*100)+"%.")

The percentage of the transcriptome they represent is 30.478896975161547%.


### 3) How many sequences with 25 or more As?
(Cond. row selection)

In [70]:
print("The number of sequences with 25 or more As is "+str(df[df.A_count>=25].shape[0])+".")

The number of sequences with 25 or more As is 5.


### 4) Clean up the dataframe (or re-run), add counts for all 4 nucl

In [71]:
nuc_count_dict={'C_count':[], 'G_count':[], 'T_count':[]}
for key in nuc_count_dict.keys():
    nuc_count_dict[key]=[count_nuc(seq, key[0]) for seq in df.Seq]
    df[key]=nuc_count_dict[key]
df.head()

,Seq,Id,Count,A_count,C_count,G_count,T_count
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,113422,31,0,0,0
1,CAGGACTCCAATATAGAGATAAGTTAATGTC,2,93,12,5,6,8
2,TATGTAATTGGTTCCAGTGTGAGTCATTAAA,3,5,9,3,7,12
3,GATATTTTCGAAAAGTGGGATTTTTTAAACC,4,88,10,3,6,12
4,CTCCATCTCAGGTATTAGAATGAATGCTTAC,5,7,9,7,5,10


(Whiz kid corner: a function returning a function)

### 5) Add a %GC column

In [72]:
df['GC_percentage']=((df['G_count']+df['C_count'])/(df['A_count']+df['T_count']+df['G_count']+df['C_count'])*100)

In [73]:
df.describe()

,Id,Count,A_count,C_count,G_count,T_count,GC_percentage
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000
mean,2000.500000,93.162000,9.427000,7.058500,6.840750,7.67375,44.836290
std,1154.844867,1804.997654,3.637922,3.313732,3.118464,3.19654,13.908145
min,1.000000,5.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,1000.750000,7.000000,7.000000,5.000000,5.000000,5.00000,35.483871
50%,2000.500000,14.000000,9.000000,7.000000,7.000000,7.00000,45.161290
75%,3000.250000,42.000000,12.000000,9.000000,9.000000,10.00000,54.838710
max,4000.000000,113422.000000,31.000000,21.000000,20.000000,20.00000,87.096774


### 6) And find the 10 sequences with highest GC content. How many reads do they represent?
(as a bonus, store this result in a new dataframe with only columns: Seq, Id, Count and %GC. You might need a few extra "tricks" with .loc[:,["Col1", "Col2"])

In [74]:
(df.sort_values(by='GC_percentage', ascending=False)).head(10)

,Seq,Id,Count,A_count,C_count,G_count,T_count,GC_percentage
1735,CTGCCCGCGCCCGCCGCCCAGGACCCCGCAC,1736,6,3,19,8,1,87.096774
963,CCGCGCCGCCCGGGCACCATGGCGGGGAAGG,964,7,4,12,14,1,83.870968
1751,CGGCGGTTGGCGGGGCACCACGGGAGGGGCC,1752,19,3,9,17,2,83.870968
233,ACCCGGCGCCCGGCCAGTCCTGCGCGTCCCC,234,38,2,17,9,3,83.870968
1508,ACGCACCCCTCCCCGGCCTGGGCGGCGGCGA,1509,72,3,15,11,2,83.870968
3021,GGGTCCGGCGCCGCCGGCTGCGGCTTCGCGA,3022,21,1,12,14,4,83.870968
1222,CTGCGGGGGGCCTGCGGAGACGGCGCCCGCA,1223,5,3,11,15,2,83.870968
3390,GCACGGGCGAAGGGGCCGCGGCCGCATGCCC,3391,64,4,12,14,1,83.870968
2021,CAGCCAGGGCGCCAGCCCCCAGCCCCAGCAC,2022,56,6,17,8,0,80.645161
2587,GCTCCCGGGGGAAGGCCCCCCCGCCAGATCC,2588,5,4,15,10,2,80.645161


In [75]:
new_df_GCpercentage=df[['Seq', 'Id', 'Count','GC_percentage']]

In [76]:
#The number of reads that they represent
nb_reads=(df.sort_values(by='GC_percentage', ascending=False)).head(10).Count.sum()
print("The number of reads that they represent is " +str(nb_reads)+ ".")

The number of reads that they represent is 293.


### 7) How many sequences with ≥ 50%GC? What is the %GC of all the sequences together? How many sequence have %GC above the expected value?

In [77]:
print("The number of sequences with a GC percentage over 50 is "+ 
      str(new_df_GCpercentage[new_df_GCpercentage.GC_percentage>=50].shape[0])+".")

The number of sequences with a GC percentage over 50 is 1453.


In [78]:
#%GC of all the sequences together
#add a column length of sequence
#df['Seq_Length']=df.apply(lambda row : len(row.Seq), axis=1) useless but cool

#Find the %GC of everything combined
total_GC_perc=((df['G_count']+df['C_count']).sum()/(df['A_count']+df['T_count']+df['G_count']+df['C_count']).sum()*100)

print("The GC percentage on the whole file is "+ str(total_GC_perc)+".")

The GC percentage on the whole file is 44.836290322580645.


In [79]:
#Number of sequences that have a GC above expected value
print("The number of sequences that have a GC above expected value "+ 
      str(new_df_GCpercentage[new_df_GCpercentage.GC_percentage>=total_GC_perc].shape[0])+".")

The number of sequences that have a GC above expected value 2104.


### (*For fun*): Which sequence would form the longest stem linking the 5' and 3' extremities?
(Answer: ATGAATTGAGTTGTGTCCCCCCAAAATTCAT, 7 base pairs, line number 2827)

In [80]:
nuc_match={'A':'T', 'C':'G', 'T':'A', 'G':'C'}

def stemlinklength(seq):
    counter=0
    n=len(seq)
    while nuc_match[seq[counter]]==seq[n-counter-1] and n-counter-1>counter:
        counter+=1
    return counter

stemlinklength('AAATTT')

3

In [81]:
df['Stem_link_length']=df.apply(lambda row : stemlinklength(row.Seq), axis=1) 

In [82]:
df_sorted_by_stem_link_length=df.sort_values(by='Stem_link_length', axis=0, ascending=False)

In [83]:
df_sorted_by_stem_link_length

,Seq,Id,Count,A_count,C_count,G_count,T_count,GC_percentage,Stem_link_length
2827,ATGAATTGAGTTGTGTCCCCCCAAAATTCAT,2828,12,9,7,5,10,38.709677,7
3956,GCTGGCAGAGGGCGCGCCAACAGCCGCCAGC,3957,10,6,12,12,1,77.419355,6
2022,GGCTCGAACCACCTCTTCTCTGAGCCGAGCC,2023,132,5,13,7,6,64.516129,6
2115,GAGGGAGTGGCGACACCACCACAGTGCCCTC,2116,16,7,11,10,3,67.741935,5
915,CCTTACTTCTGTTAATGGTGTGATAGAAAGG,916,5,8,4,8,11,38.709677,4
...,...,...,...,...,...,...,...,...,...
1545,ACGTAGCATCCTCCAGGGAGTGAGTTGGAAC,1546,12,8,7,10,6,54.838710,0
1546,AGTGAACCAGTATTTTCCAGATGACCAATGC,1547,17,10,7,6,8,41.935484,0
1548,AACTTGTTTTACAAATCTGGGTGCTTCTGTA,1549,7,7,5,6,13,35.483871,0
1549,GAAGCAAATTAATATTGTATCAGGTTTTGAA,1550,5,12,2,6,11,25.806452,0
